# Giriş işlemleri

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch
import pandas as pd
from tqdm.auto import tqdm
from typing import Optional, Tuple
from peft import PeftModel
import random
import numpy as np

In [ ]:
def is_colab() -> bool:
    """
    Google Colab ortamında çalışıp çalışmadığını kontrol eden fonksiyon.

    Args:
        None

    Returns:
        bool: Eğer kod Google Colab'da çalışıyorsa True, aksi halde False döndürür
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    """
    Eğer kod Google Colab ortamında çalışıyorsa, Google Drive'ı bağlar ve
    kök dizini Google Drive içindeki "turkish_gpt2_finetuning" klasörü olarak ayarlar.
    """
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive'ı Colab ortamına bağlar
    kok_dizin = "/content/drive/MyDrive/turkish_gpt2_finetuning"  # Drive içindeki çalışma klasörünü belirler
else:
    """
    Eğer kod yerel bir ortamda çalışıyorsa, kök dizini mevcut çalışma dizini olarak ayarlar.
    """
    kok_dizin = os.getcwd()  # Mevcut çalışma dizinini alır

# Belirlenen kök dizini kullanıcıya bilgi olarak gösterir
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
veri_kumesi_yolu = os.path.join(kok_dizin, "sinama_sorulari.csv")
sonuc_dizini     = os.path.join(kok_dizin, "sonuclar")
# Model kaydetme dizinleri
# gpt4o verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_medium_gpt4o")
gpt2_large_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_large_gpt4o")

# deepseek verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_medium_deepseek")
gpt2_large_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_large_deepseek")

# Model adı tanımlamaları
gpt2_medium_model_adi = "ytu-ce-cosmos/turkish-gpt2-medium"
gpt2_large_model_adi = "ytu-ce-cosmos/turkish-gpt2-large"

model_bilgileri = [
    (gpt2_medium_model_adi, "gpt2_medium_gpt4o" , gpt2_medium_kaydetme_dizini_gpt4o),
    (gpt2_medium_model_adi, "gpt2_medium_deepseek", gpt2_medium_kaydetme_dizini_deepseek),
    (gpt2_large_model_adi , "gpt2_large_gpt4o" , gpt2_large_kaydetme_dizini_gpt4o),
    (gpt2_large_model_adi , "gpt2_large_deepseek", gpt2_large_kaydetme_dizini_deepseek),
]

soru_sutunu = "soru"

# Özel token tanımlamaları (sınama biçimi için)
soru_baslangic = "<SORU>"
soru_bitis = "</SORU>"
cevap_baslangic = "<CEVAP>"
cevap_bitis = "</CEVAP>"

# Fonksiyonlar

## Soruyu girdi biçimine çevirme fonksiyonu

In [ ]:
def soruyu_girdi_bicimine_cevir(soru: str) -> str:
    """
    Verilen ham soruyu, dil modeline uygun istem (prompt) biçimine dönüştürür.

    Fonksiyon; dışarıda global olarak tanımlı özel etiketleri (<SORU>, </SORU>,
    <CEVAP>) kullanarak aşağıdaki kalıpta bir metin hazırlar:

        "<SORU> {soru} </SORU> <CEVAP> "

    Model tarafında generate() çağrısı sırasında, </CEVAP> veya EOS token’ı
    üretildiğinde üretim sonlandırılır ve yalnızca cevaba ait kısım alınır.
    Böylece sorunun başı ve bitişi ile cevabın başlangıcı tutarlı bir şekilde
    modellenmiş olur.

    Args:
        soru (str): Kullanıcıdan gelen ham soru metni. Baş-son boşlukları
                    fonksiyon içinde temizlenir (str.strip()).

    Returns:
        str: Dil modeline beslenmeye hazır, etiketlenmiş prompt dizgesi.

    Örnek:
        >>> soruyu_girdi_bicimine_cevir("Türkiye'nin başkenti neresidir?")
        '<SORU> Türkiye'nin başkenti neresidir? </SORU> <CEVAP> '
    """

    # ---------------------------------------------------------- #
    # 1) Giriş temizliği
    # ---------------------------------------------------------- #
    # Sorunun başında / sonunda fazladan boşluk varsa yok sayılır.
    soru = soru.strip()

    # ---------------------------------------------------------- #
    # 2) Prompt kalıbını oluştur
    # ---------------------------------------------------------- #
    # Global sabitler:
    #   soru_baslangic = "<SORU>"
    #   soru_bitis     = "</SORU>"
    #   cevap_baslangic= "<CEVAP>"
    #
    # Model, </CEVAP> token’ına (veya eos_token_id’ye) ulaştığında duracağı
    # için cevap_bitis etiketine şimdi ihtiyaç yok; generate sonrasında
    # kesilecek.
    prompt = f"{soru_baslangic} {soru} {soru_bitis} {cevap_baslangic} "

    # ---------------------------------------------------------- #
    # 3) Sonucu döndür
    # ---------------------------------------------------------- #
    return prompt


## Modeli yükleme fonksiyonu

In [ ]:
def model_yukle(
    ince_ayari_yolu: str,
    *,
    base_model_adi: str,
    dtype: str = "auto",
) -> Tuple[AutoModelForCausalLM, AutoTokenizer]:
    """
    İnce-ayarlı (fine-tuned) bir dil modelini veya yalnızca LoRA/adapter dosyalarını
    otomatik olarak yükler.

    • Eğer `ince_ayari_yolu` klasöründe TAM model ağırlıkları bulunuyorsa
      (pytorch_model.bin veya model.safetensors) → doğrudan modeli yükler.
    • Klasörde yalnızca LoRA/adapter dosyaları varsa →
      1) Önce `base_model_adi` ile temel (base) modeli indirir/yükler,
      2) Daha sonra adapter’ı temel modele uygular ve LoRA katmanlarını
         `merge_and_unload()` ile birleşik hâle getirip hafızadan çıkarır.

    Args:
        ince_ayari_yolu (str):
            İnce-ayarlı ağırlıkların (veya LoRA dosyalarının) yer aldığı klasör yolu.
        base_model_adi (str):
            Sadece adapter varsa indirilmesi gereken temel modelin HuggingFace hub adı
            veya yerel yolu. Bu parametre *keyword-only* olarak tanımlıdır;
            fonksiyonu çağırırken `base_model_adi="..."` şeklinde belirtilmelidir.
        dtype (str, optional):
            Kullanılacak PyTorch *dtype*’ı.
            "auto"  → Cihaz yeteneğine göre bf16 / fp16 / fp32 seçilir (varsayılan).
            "fp16"  → Zorla float16.
            "bf16"  → Zorla bfloat16.
            Diğer   → float32.
            NOT: Burada yalnızca hesaplamalarda kullanılacak veri tipi belirlenir;
            model mimarisi değişmez.

    Returns:
        Tuple[AutoModelForCausalLM, AutoTokenizer]:
            0. indis → Yüklenen (varsa birleştirilmiş) dil modeli
            1. indis → İnce-ayarlı klasörden gelen, pad token’ı ayarlanmış tokenizer
    """

    # ------------------------------------------------------------------
    # 1) dtype SEÇİM MANTIĞI
    # ------------------------------------------------------------------
    # "auto" modunda:
    #   • Eğer CUDA GPU varsa, cihazın "major compute capability" değeri >= 8 ise
    #     (ör: Ampere veya üstü) bfloat16, aksi takdirde float16 kullanılır.
    #   • CUDA yoksa zorunlu olarak float32'e düşer.
    # "fp16"/"bf16" → kullanıcı tarafından belirtilen dtype alınır.
    # Diğer tüm değerler → float32.
    # ------------------------------------------------------------------
    if dtype == "auto":
        if torch.cuda.is_available():
            major_cc = torch.cuda.get_device_capability(0)[0]  # (major, minor) tuple → major
            kull_dtype = torch.bfloat16 if major_cc >= 8 else torch.float16
        else:
            kull_dtype = torch.float32
    elif dtype == "fp16":
        kull_dtype = torch.float16
    elif dtype == "bf16":
        kull_dtype = torch.bfloat16
    else:
        kull_dtype = torch.float32

    print(f"\n[MODEL] '{ince_ayari_yolu}' yükleniyor (dtype={kull_dtype})...")

    # ------------------------------------------------------------------
    # 2) KLASÖRDE TAM MODEL AĞIRLIKLARI VAR MI?
    # ------------------------------------------------------------------
    #   • pytorch_model.bin  → Klasik PyTorch checkpoint
    #   • model.safetensors → Daha güvenli / hızlı safetensors biçimi
    # Varlıklardan herhangi biri bulunursa `tam_model_var` True olur.
    # ------------------------------------------------------------------
    tam_model_var = any(
        os.path.isfile(os.path.join(ince_ayari_yolu, f))
        for f in ("pytorch_model.bin", "model.safetensors")
    )


    # ------------------------------------------------------------------
    # 3) TOKENIZER → DAİMA İNCE-AYARLI KLASÖRDEN
    # ------------------------------------------------------------------
    tokenizer = AutoTokenizer.from_pretrained(ince_ayari_yolu, use_fast=True)


    # ------------------------------------------------------------------
    # 4) MODELİ YÜKLE / BİRLEŞTİR
    # ------------------------------------------------------------------
    if tam_model_var:
        # ► Klasördeki TAM modeli doğrudan belleğe al.
        print("[MODEL] Tam model yükleniyor...")
        model = AutoModelForCausalLM.from_pretrained(
            ince_ayari_yolu,
            torch_dtype=kull_dtype,
            device_map="auto",   # HuggingFace → GPU RAM’ine otomatik dağıtım
        )
    else:
        # ► Yalnızca adapter varsa:
        #   a) Base modeli getir,
        #   b) Adapter’ı üzerine uygula,
        #   c) merge_and_unload() ile LoRA ağırlıklarını kalıcı şekilde birleştir.
        print("[MODEL] LoRA ağırlıkları yükleniyor...")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_adi,
            torch_dtype=kull_dtype,
            device_map="auto",
        )
        base_model.resize_token_embeddings(len(tokenizer))
        model = PeftModel.from_pretrained(base_model, ince_ayari_yolu)
        model = model.merge_and_unload()  # Inference’ta tek parça model kullanımı için

    # Eğer tokenizer’da pad_token tanımlı değilse, EOS token’ını PAD olarak ayarla.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # ------------------------------------------------------------------
    # 5) FONKSİYON ÇIKIŞI
    # ------------------------------------------------------------------
    return model, tokenizer

## Modelden cevabı alma fonksiyonu

In [ ]:
def cevap_uret(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    prompt: str,
    *,
    max_new_tokens: int = 200,           # Üretilecek azamî yeni token sayısı
    temperature: float = 0.8,            # 1’den küçük => daha tutarlı / deterministik
    top_p: float = 0.90,                 # Nucleus Sampling oranı
    typical_p: float = 0.95,             # Tekrarı bastırmaya yardımcı olur
    no_repeat_ngram_size: int = 3,       # Belirtilen uzunluktaki n-gram’lerin tekrarını engeller
    repetition_penalty: float = 1.15,    # Tekrarı cezalandırma katsayısı
    eos_token_id: Optional[int] = None,  # Erken durdurma için özel <EOS> belirteci
    seed: int = 571                      # Sonuçların tekrarlanabilirliği için rasgelelik tohumu
) -> str:
    """
    Örneklemeye (``do_sample=True``) dayalı olarak,
    girdiye (“prompt”) uygun, aşırı uzamayan bir yanıt üretir.

    Args:
        model (AutoModelForCausalLM): Yanıtı üretecek, nedensel dil modeli.
        tokenizer (AutoTokenizer): Modele uygun belirteçleştirici (tokenizer).
        prompt (str): Modelle beslenecek giriş metni.
        max_new_tokens (int, optional): Üretilecek en fazla yeni token sayısı.
        temperature (float, optional): Sıcaklık; 1’den düşük değerler çıktıyı tutarlı kılar.
        top_p (float, optional): Nucleus Sampling için kümülatif olasılık eşiği.
        typical_p (float, optional): “Typical sampling” olasılık eşiği (tekrarı bastırma).
        no_repeat_ngram_size (int, optional): Tekrarı engellenecek n-gram uzunluğu.
        repetition_penalty (float, optional): Tekrar eden dizileri cezalandırma katsayısı.
        eos_token_id (Optional[int], optional): Erken durdurma <EOS> token kimliği.
        seed (int, optional): Rasgele sayı üreteci tohum değeri (deterministiklik).

    Returns:
        str: ``prompt`` sonrası üretilen, temizlenmiş yanıt metni.
    """

    # ---------------------------------------------------------------------
    # 1. <EOS> belirtecinin kimliğini ayarla
    # ---------------------------------------------------------------------
    # Kullanıcı belirli bir `eos_token_id` vermediyse, tokenizer’dan
    # özel olarak “</CEVAP>” metninin kimliğini alıyoruz.  
    # Bu özel belirteç yanıtın bittiği yeri işaretlemek için kullanılır.
    # ---------------------------------------------------------------------
    if eos_token_id is None:
        eos_token_id = tokenizer.convert_tokens_to_ids("</CEVAP>")

    # ---------------------------------------------------------------------
    # 2. Tohumları (seed) ayarla
    # ---------------------------------------------------------------------
    # GPU veya CPU fark etmeksizin tekrarlanabilir sonuçlar almak için
    # Python’un, NumPy’nin ve PyTorch’un rasgele sayı üreteçlerine aynı
    # tohum değerini aktarırız.
    # ---------------------------------------------------------------------
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        # Birden fazla CUDA çekirdeği varsa hepsine uygula
        torch.cuda.manual_seed_all(seed)

    # ---------------------------------------------------------------------
    # 3. Girdiyi tensöre çevir
    # ---------------------------------------------------------------------
    # `tokenizer` ile metin → token kimliği; ardından `.to(model.device)`
    # ile modelin bulunduğu cihaza (CPU / GPU) taşırız.
    # ---------------------------------------------------------------------
    giris_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len: int = giris_ids["input_ids"].shape[-1]   # Prompt uzunluğu

    # ---------------------------------------------------------------------
    # 4. Metin üretimi (sampling tabanlı)
    # ---------------------------------------------------------------------
    # `model.generate` ile istenen parametreler eşliğinde üretim yapılır.
    # `do_sample=True` → Greedy yerine olasılığa dayalı seçim.
    # `early_stopping=True` → İlk <EOS> token’ında dur.
    # ---------------------------------------------------------------------
    cikti_ids = model.generate(
        **giris_ids,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        typical_p=typical_p,
        max_new_tokens=max_new_tokens,
        no_repeat_ngram_size=no_repeat_ngram_size,
        repetition_penalty=repetition_penalty,
        eos_token_id=eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        early_stopping=True
    )

    # ---------------------------------------------------------------------
    # 5. Sadece “prompt” sonrasını al
    # ---------------------------------------------------------------------
    # Model tüm diziyi (prompt + yanıt) döndürür. Biz yalnızca
    # yeni üretilen kısmı (`input_len:`) dekode ederiz.
    # ---------------------------------------------------------------------
    yeni_tokenlar: torch.Tensor = cikti_ids[0][input_len:]
    yanit: str = tokenizer.decode(yeni_tokenlar, skip_special_tokens=True)

    # ---------------------------------------------------------------------
    # 6. Fazlalıkları temizle ve döndür
    # ---------------------------------------------------------------------
    # “</CEVAP>” dizesiyle karşılaşıldığında orada kesilir, 
    # uçlardaki boşluklar temizlenir.
    # ---------------------------------------------------------------------
    return yanit.split("</CEVAP>")[0].strip()

# Sonuç üretme kısmı

## Veri kümesi oku

In [ ]:
df_sorular = pd.read_csv(veri_kumesi_yolu)
print(f"{len(df_sorular)} soru yüklendi.")

## Her model için sonuç üret ve kaydet

In [ ]:
# sonuçlar dizinini oluştur
os.makedirs(sonuc_dizini, exist_ok=True)

In [ ]:
# --------------------------------------------------------------------------- #
#  Tüm modelleri tek tek dolaşarak, her birine karşılık sınama sorularının
#  cevaplarını üretip .csv dosyasına kaydeden ana döngü
# --------------------------------------------------------------------------- #

for model_bilgisi in model_bilgileri:
    model_etiket = model_bilgisi[1]
    tuned_folder = model_bilgisi[2]
    base_name = model_bilgisi[0]
    # ----------------------------------------------------------------------- #
    # 1) Çıktı dosyası yolu oluşturma
    # ----------------------------------------------------------------------- #
    # Örneğin `veri_kumesi = "gpt2_medium_gpt4o"` ise:
    #   cikti_yolu = "<kok_dizin>/sonuclar/gpt2_medium_gpt4o.csv"
    cikti_yolu = os.path.join(sonuc_dizini, f"{model_etiket}.csv")

    # Aynı model için çıktı dosyası daha önce oluşturulmuşsa hesaplama
    # tekrarlanmasın; zaman ve kaynak kazancı sağlanır.
    if os.path.exists(cikti_yolu):
        print(f"[SKIP] '{cikti_yolu}' zaten var, geçiliyor.")
        continue

    # ---- 2) Model + Tokenizer yükle  ----------------------------
    model, tokenizer = model_yukle(
        tuned_folder,          # ince-ayarlı klasör
        base_model_adi=base_name   # hangi temel modelden türedi
    )

    # `model.eval()`  ⇒  Eğitim modunu kapat, tahmin (inference) moduna geç
    #                  (dropout, layer norm gibi davranışlar sabitlenir).
    model.eval()

    # ----------------------------------------------------------------------- #
    # 3) Soru listesi üzerinde döngü – her bir sorudan cevap üret
    # ----------------------------------------------------------------------- #
    cevaplar = []  # Üretilen cevapları tutacak liste

    # tqdm → ilerleme çubuğu; iterasyon durumunu terminalde gösterir
    for soru in tqdm(df_sorular["soru"], desc=model_etiket):
        # Soruyu, modelin eğitildiği özel etiketli prompt biçimine çevir
        prompt = soruyu_girdi_bicimine_cevir(soru)

        # cevap_uret() → deterministik (greedy) generate çağrısı yapar
        cevap  = cevap_uret(model, tokenizer, prompt)

        # Listeye ekle
        cevaplar.append(cevap)

    # ----------------------------------------------------------------------- #
    # 4) Sonuç DataFrame’i oluştur ve diske kaydet
    # ----------------------------------------------------------------------- #
    #   • Başlangıçta okunan df_sorular DataFrame'inin aynısını kopyalayıp
    #     yanına yeni bir "cevap" sütunu ekliyoruz.
    df_sonuclar = df_sorular.copy()
    df_sonuclar["cevap"] = cevaplar

    # .csv olarak kaydet → Unicode karakterler için encoding="utf-8"
    df_sonuclar.to_csv(cikti_yolu, index=False, encoding="utf-8")
    print(f"[OK] {model_etiket} sonuçları '{cikti_yolu}' dosyasına kaydedildi.")

    # ----------------------------------------------------------------------- #
    # 5) Bellek temizliği
    # ----------------------------------------------------------------------- #
    # Bir sonraki model yüklenmeden önce GPU belleğini boşaltmak kritik.
    #   • `del model`           → Python referansını sil
    #   • `torch.cuda.empty_cache()` → PyTorch'un ayrılmış fakat kullanılmayan
    #                                   GPU belleğini boşalt
    del model
    torch.cuda.empty_cache()